In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Kaggle/train.csv')
dataset.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
dataset.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [4]:
sentiment = dataset['comment_text'].values
y_list = ["toxic","severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = dataset[y_list].values
sentiment

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [5]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(sentiment))

In [6]:
from keras.preprocessing import text
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq = tokenizer.texts_to_sequences(sentiment)
pad = pad_sequences(seq, maxlen=100)


In [7]:
test = pd.read_csv('/content/drive/MyDrive/preprocessed_Reddit_Data_1.csv')

columns_to_keep = ['Title']
predict_data = test[columns_to_keep]

In [8]:
test = test['Title'].values
test_seq = tokenizer.texts_to_sequences(test)
test_pad = pad_sequences(test_seq, maxlen=100)

In [9]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers


def model_add():
    inputs = Input(shape=(100, ))
    x = Embedding(20000, 128)(inputs)
    x = Bidirectional(LSTM(50))(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
model = model_add()
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 128)          2560000   
                                                                 
 bidirectional (Bidirectiona  (None, 100)              71600     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                             

In [10]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
model.fit(pad, y, batch_size=32, epochs=2, validation_split=0.1, callbacks=early)


Epoch 1/2
4488/4488 [==============================] - 157s 33ms/step - loss: 0.0653 - accuracy: 0.9401 - val_loss: 0.0519 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 66s 15ms/step - loss: 0.0462 - accuracy: 0.9740 - val_loss: 0.0516 - val_accuracy: 0.9939


In [11]:
model.save('/content/drive/MyDrive/LSTM_model')

In [25]:
y_test = model.predict([test_pad], batch_size=1024, verbose=1)

threshold = 0.0001  # Adjust this threshold as needed

# Apply threshold and convert to 0 or 1
y_test[y_test >= threshold] = 1
y_test[y_test < threshold] = 0

# Convert the array to integer type
y_test = y_test.astype(int)
# Create a new DataFrame with the predicted labels
predict_labels_df = pd.DataFrame(y_test, columns=y_list)

# Combine the original DataFrame with the predicted labels
predict_df = pd.read_csv('/content/drive/MyDrive/preprocessed_Reddit_Data_1.csv')
predict_df = pd.concat([predict_df, predict_labels_df], axis=1)

# Save the DataFrame to a CSV file
predict_df.to_csv('/content/drive/MyDrive/predict_lstm.csv', index=False)

1/1 [==============================] - 0s 37ms/step


In [26]:
predict_df

,Unnamed: 0,Title,Author,id,clean_title,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,UkrainianConflict Discussion Megathread,humanlikecorvus,y7gz80,"['ukrainianconflict', 'discussion', 'megathread']",1,1,1,1,1,1
1,1,Zelenskyy survives over 12 assassination attem...,Far-Childhood9338,10e17wq,"['zelenskyy', 'survives', 'assassination', 'at...",1,0,0,0,0,0
2,2,In the first round of presidential elections i...,RevealDisinfo,10digs3,"['first', 'round', 'presidential', 'elections'...",0,0,0,0,0,0
3,3,"A further 20,000 Ukrainian recruits will be tr...",tedwja,10dv085,"['ukrainian', 'recruits', 'trained', 'uk', 'ye...",1,0,1,0,0,0
4,4,"Zelensky: ""Tanks, APCs and artillery are exact...",zizp,10duei9,"['zelensky', 'tanks', 'apcs', 'artillery', 'ex...",1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
971,971,Hundreds of US military vehicles arrive in Dut...,Standard_Spaniard,109fbyj,"['hundreds', 'us', 'military', 'vehicles', 'ar...",1,0,0,0,0,0
972,972,BREAKING: Poland will deliver a company of Leo...,rulepanic,1096adh,"['breaking', 'poland', 'deliver', 'company', '...",0,0,0,0,0,0
973,973,Russian airline aircraft suffer massive breakd...,Breech_Loader,109envz,"['russian', 'airline', 'aircraft', 'suffer', '...",1,0,1,0,1,1
974,974,"The Russian Federation declared that it ""has t...",RevealDisinfo,1095iye,"['russian', 'federation', 'declared', 'right',...",1,0,0,0,0,0


In [27]:
model.evaluate(test_pad, y_test, batch_size=1024, verbose=1)


1/1 [==============================] - 0s 43ms/step - loss: 2.5823 - accuracy: 1.0000


[2.5822594165802, 1.0]